## Connect Google Colab with Google Drive



In [13]:
link = '/Users/dhrumilshah/Desktop/works/covtype.csv'

In [14]:
import pandas as pd


In [15]:

df3 = pd.read_csv('/Users/dhrumilshah/Desktop/works/covtype.csv')# Dataset is now stored in a Pandas Dataframe

## Importing Required Libraries

In [16]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler, MinMaxScaler, maxabs_scale, MaxAbsScaler
import time

## Class Encoder and Decoder

In [17]:
class Encoder(nn.Module):
    def __init__(self, inputt, hidden):
        super(Encoder, self).__init__()
        self.inputt = inputt
        self.hidden = hidden
        self.fca = nn.Linear(inputt, inputt)
        self.fc1 = nn.Linear(inputt, hidden)

    def forward(self, x, thresh):
        x = x.view(-1, self.inputt)
        x_a = F.sigmoid(self.fca(x))
        x_a = (x_a > thresh).float()
        x = torch.mul(x, x_a)
        return F.relu(self.fc1(x))

class Decoder(nn.Module):
    def __init__(self, inputt, hidden):
        super(Decoder, self).__init__()
        self.inputt = inputt
        self.hidden = hidden
        self.fc1 = nn.Linear(inputt, hidden)

    def forward(self, x):
        return F.relu(self.fc1(x))

class AutoEncoder(nn.Module):
    def __init__(self, inputt=54, hidden=200):
        super(AutoEncoder, self).__init__()
        self.inputt = inputt
        self.hidden = hidden
        self.fc1 = Encoder(inputt, hidden)
        self.fc2 = Decoder(hidden, inputt)

    def forward(self, x, thresh):
        return self.fc2(self.fc1(x, thresh))

class Classifier(nn.Module):
    def __init__(self, inputt=200,out=7):
        super(Classifier, self).__init__()
        self.inputt = inputt
        self.out = out
        self.fc1 = nn.Linear(inputt, out)

    def forward(self, x):
        return F.softmax(self.fc1(x))

def kl_divergence(p, q):

    p = F.softmax(p)
    q = F.softmax(q)

    return torch.sum(p * torch.log(p / q)) + torch.sum((1 - p) * torch.log((1 - p) / (1 - q)))


## model

In [18]:
def train(model, model2, device, train_loader, optimizer, optimizer2, epoch, log_interval, sparsity_param, thresh, inp_size, batch_size):
    model.train()
    eff_number_of_sensors = []
    train_loss = 0
    correct = 0
    for batch_idx, (data, label) in enumerate(train_loader):  #Itererate over the training data in batches

        data, label = data.to(device), label.to(device) # copy train data to either GPU or CPU
        optimizer.zero_grad() # Set grad to zero
        optimizer2.zero_grad() # Set grad to zero
        output = model(data, thresh)  # forward propagation
        target = data.view(-1, inp_size)
        attention = F.sigmoid(model.fc1.fca(data.view(-1, inp_size)))
        encoded = model.fc1(torch.mul(data.view(-1, inp_size),attention), thresh)
        ########## classify the encodings #######
        output2 = model2(encoded)  # forward propagation
        pred = output2.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(label.view_as(pred)).sum().item()
        loss2 = F.nll_loss(output2, label) # Negative Log likelihood loss
        loss2.backward(retain_graph=True) # Error Backpropagation
        optimizer2.step() # update weights
        ###############################################
        rho_hat = torch.sum(encoded, dim=0, keepdim=True)/len(target)
        loss = F.mse_loss(output, target) + 5*kl_divergence(sparsity_param,  rho_hat.cpu()).to(device) + 0.001*torch.sum(attention.to(device))/(batch_size *inp_size)
        train_loss += F.mse_loss(output, target, reduction='sum').item()

        eff_number_of_sensors.append((torch.sum((attention>thresh).float())/(len(target))).cpu().detach().numpy())
        loss.backward() # Error Backpropagation
        optimizer.step() # update weights
        if batch_idx % log_interval == 0:  # for printing loss
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    acc = 100. * correct / len(train_loader.dataset)
    print('\nTrain accuracy:', acc)
    train_loss /= (len(train_loader)*inp_size)
    print('\nTrain set: Average loss:', train_loss)
    return train_loss, np.mean(eff_number_of_sensors), acc

## Testing Model

In [19]:
def test(model, model2, device, test_loader, thresh, inp_sz):
    model.eval()
    test_loss = 0
    test_loss2 = 0
    correct = 0
    eff_number_of_sensors_test = []
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.to(device), label.to(device)  # copy test data to either GPU or CPU
            output = model(data, thresh)  # forward propagation
            target = data.view(-1, inp_sz)
            start = time.time()
            attention = F.sigmoid(model.fc1.fca(data.view(-1, inp_sz)))
            encoded = model.fc1(torch.mul(data.view(-1, inp_sz), attention), thresh)
            #############################################################
            ## Classify the encodings #################################
            output2 = model2(encoded)  # forward propagation
            end = time.time()
            diff = end - start
            test_loss2 += F.nll_loss(output2, label, reduction='sum').item()  # sum up batch loss
            pred = output2.max(1, keepdim=True)[1]  # get the index of the max log-probability
            correct += pred.eq(label.view_as(pred)).sum().item()
            ############################################################
            eff_number_of_sensors_test.append(
                (torch.sum((attention > thresh).float()) / (len(target))).cpu().detach().numpy())
            test_loss += F.mse_loss(output, target, reduction='sum').item()  # sum up batch loss
    #             print(test_loss)
    test_loss /= (len(test_loader) * inp_sz)
    acc = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss:', test_loss)
    print('\nTest accuracy:', acc)

    return test_loss, np.mean(eff_number_of_sensors_test), acc

## data Normalize

In [20]:
def sample(x, n):
    return x.iloc[random.sample(x.index, n)]


def balance_data(df):
    balanced_data = pd.DataFrame()  # Initialize an empty DataFrame
    for i in range(1, 8):
        print(i)
        # Select the subset of the DataFrame where `Cover_Type` is equal to `i`
        cond = df['Cover_Type'] == i
        subset = df[cond].dropna()

        # Sample 2747 rows from each subset
        subset = subset.sample(n=2747)

        # Concatenate the subset into the balanced_data DataFrame
        balanced_data = pd.concat([balanced_data, subset], ignore_index=True)

    return balanced_data


## get Data

In [21]:

def getData(filename, balance=False):
    chunksize = 1200000
    flag = 1
    for data in pd.read_csv(filename, sep=",", chunksize=chunksize, on_bad_lines='skip'):
        while flag < 2 and chunksize < 1200000:
            print(data)
        flag += 1
    print(list(data.columns.values))

    if balance:
        data = balance_data(data)
    return data



def convert2Tensor(trainData, trainLabel, testData, testLabel, batch_size, kwargs):
    # training data
    tensor_x = torch.stack([torch.Tensor(i) for i in trainData])  # transform to torch tensors
    tensor_y = torch.Tensor(trainLabel).long()
    my_dataset = utils.TensorDataset(tensor_x, tensor_y)  # create your datset
    train_loader = utils.DataLoader(my_dataset, batch_size=batch_size, shuffle=True, **kwargs)  # create your dataloader

    # test data
    tensor_x = torch.stack([torch.Tensor(i) for i in testData])  # transform to torch tensors
    tensor_y = torch.Tensor(testLabel).long()
    my_dataset = utils.TensorDataset(tensor_x, tensor_y)  # create your dataset
    test_loader = utils.DataLoader(my_dataset, batch_size=batch_size, shuffle=False, **kwargs)  #

    return train_loader, test_loader

In [22]:
def normalize_data(x_train, x_test):
    # training
    norm_tcolumns = x_train[x_train.columns[:10]]  # only the first ten columns need normalization, the rest is binary
    #     scaler = MinMaxScaler(copy=True, feature_range=(0, 1)).fit(norm_tcolumns.values)
    scaler = MaxAbsScaler(copy=True).fit(norm_tcolumns.values)
    scaledf = scaler.transform(norm_tcolumns.values)
    training_examples = pd.DataFrame(scaledf, index=norm_tcolumns.index,
                                     columns=norm_tcolumns.columns)  # scaledf is converted from array to dataframe
    x_train.update(training_examples)

    # validation
    norm_vcolumns = x_test[x_test.columns[:10]]
    vscaled = scaler.transform(norm_vcolumns.values)  # this scaler uses std and mean of training dataset
    validation_examples = pd.DataFrame(vscaled, index=norm_vcolumns.index, columns=norm_vcolumns.columns)
    x_test.update(validation_examples)
    return x_train, x_test

## Train and Test Split

In [23]:
def split_data(data):
    # splitting the data
    msk = np.random.rand(len(data)) < 0.8
    yy = ['Cover_Type']

    train = data[msk]
    test = data[~msk]
    #     print(train.head())
    #     print(test.head())
    x_train = train[train.columns[:train.shape[1] - 1]]  # all columns except the last are x variables
    y_train = train[yy[0]].tolist()  # the last column as y variable
    x_test = test[test.columns[:test.shape[1] - 1]]
    y_test = test[yy[0]].tolist()  # the last column as y variable

    y_train = [0 if i == 7 else i for i in y_train]  # replace label 7 with 0
    y_test = [0 if i == 7 else i for i in y_test]  # replace label 7 with 0

    return x_train, y_train, x_test, y_test

def main():
    #################################################
    ### Training settings ###########################

    inp_size = 54
    hidden_size = 50
    numClasses = 7
    batch_size = 100
    test_batch_size = 50
    epochs = 800
    lr = 0.95
    seed = 1
    log_interval = 10
    use_cuda = True
    rho = 0.5
    thresh = 0.1

    param = {'inp_size': inp_size,'hidden_size': hidden_size, 'batch_size': batch_size, 'numClasses': numClasses, 'epochs': epochs, 'lr': lr, 'seed': seed, 'rho': rho, 'thresh': thresh}

    sparsity_param = torch.FloatTensor([rho for _ in range(hidden_size)]).unsqueeze(0)

    torch.manual_seed(seed)
    ########## Choosing GPU or CPU ######################
    use_cuda = False
    device = torch.device("cuda" if use_cuda else "cpu")
    #############  Data Loader ##############
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    ####Training
    ###########################################################
    filename = 'covtype.csv'
    dataset = getData(filename, balance=True)
    X_train, y_train, X_test, y_test = split_data(dataset)
    X_train, X_test = normalize_data(X_train, X_test)

    print('==============Training================')
    print('number of class 1: ', y_train.count(1))
    print('number of class 2: ', y_train.count(2))
    print('number of class 3: ', y_train.count(3))
    print('number of class 4: ', y_train.count(4))
    print('number of class 5: ', y_train.count(5))
    print('number of class 6: ', y_train.count(6))
    print('number of class 7: ', y_train.count(0))

    print('==============Testing================')
    print('number of class 1: ', y_test.count(1))
    print('number of class 2: ', y_test.count(2))
    print('number of class 3: ', y_test.count(3))
    print('number of class 4: ', y_test.count(4))
    print('number of class 5: ', y_test.count(5))
    print('number of class 6: ', y_test.count(6))
    print('number of class 7: ', y_test.count(0))

    X_train = X_train.iloc[:, :].values
    X_test = X_test.iloc[:, :].values
    train_loader, test_loader = convert2Tensor(X_train, y_train, X_test, y_test, batch_size, kwargs)
    #############################################################
    ############# Instantiate Model #############################
    model = AutoEncoder(inp_size, hidden_size).to(device)
    print(model)
    classifier = Classifier(hidden_size, numClasses).to(device)
    print(classifier)
    ######################### Define optimization #####################
    optimizer  = optim.SGD(model.parameters(), lr=lr)
    optimizer_classifier = optim.SGD(classifier.parameters(), lr=0.01)
    ###################################################################
    ### Epoch training
    loss_train_arr = []
    loss_test_arr = []
    sensors_train = []
    sensors_test = []
    test_acc = []
    train_acc = []
    prob_test = []
    acc_best = 0
    for epoch in range(1, epochs + 1):
        loss_train, no_sensors_train, acc_train = train(model, classifier, device, train_loader, optimizer,
                                                              optimizer_classifier, epoch, log_interval, sparsity_param,
                                                              thresh, inp_size, batch_size)
        loss_test, no_sensors_test, acc = test(model, classifier, device, test_loader, thresh,
                                                                     inp_size)

        ## Store Metrics
        loss_train_arr.append(loss_train)
        loss_test_arr.append(loss_test)
        sensors_train.append(no_sensors_train)
        sensors_test.append(no_sensors_test)
        test_acc.append(acc)
        train_acc.append(acc_train)

    return loss_train_arr, loss_test_arr, sensors_train, sensors_test, train_acc, test_acc

## Running Model

In [24]:
if __name__ == '__main__':
        loss_train_arr, loss_test_arr, sensors_train, sensors_test, train_acc, test_acc = main()
        print('Train Accuracy', np.max(train_acc))
        print('Test Accuracy', np.max(test_acc))

        idx = np.argmax(test_acc)
        print('index is:', idx)
        print('Number of train sensors', sensors_train[idx])
        print('Number of test sensors', sensors_test[idx])
        print('Train recon loss', loss_train_arr[idx])
        print('Test recon loss', loss_test_arr[idx])

['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Cover_Type']
1
2
3
4
5
6
7
==============Training================
number of class 1:  2202
number of

/var/folders/rd/7_9y283942j3_n1xth0wq7cr0000gn/T/ipykernel_39353/1179746991.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.83827353 0.7301092  0.84659386 ... 0.84919397 0.86713469 0.89053562]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  x_train.update(training_examples)
/var/folders/rd/7_9y283942j3_n1xth0wq7cr0000gn/T/ipykernel_39353/1179746991.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.45833333 0.25       0.00277778 ... 0.86388889 0.57222222 0.59722222]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  x_train.update(training_examples)
/var/folders/rd/7_9y283942j3_n1xth0wq7cr0000gn/T/ipykernel_39353/1179746991.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.05555

Train Epoch: 1 [0/15404 (0%)]	Loss: 0.085716
Train Epoch: 1 [1000/15404 (6%)]	Loss: 0.060752
Train Epoch: 1 [2000/15404 (13%)]	Loss: 0.052784
Train Epoch: 1 [3000/15404 (19%)]	Loss: 0.041851
Train Epoch: 1 [4000/15404 (26%)]	Loss: 0.037683
Train Epoch: 1 [5000/15404 (32%)]	Loss: 0.035786
Train Epoch: 1 [6000/15404 (39%)]	Loss: 0.034845
Train Epoch: 1 [7000/15404 (45%)]	Loss: 0.033413
Train Epoch: 1 [8000/15404 (52%)]	Loss: 0.029425
Train Epoch: 1 [9000/15404 (58%)]	Loss: 0.027436
Train Epoch: 1 [10000/15404 (65%)]	Loss: 0.027001
Train Epoch: 1 [11000/15404 (71%)]	Loss: 0.027022
Train Epoch: 1 [12000/15404 (77%)]	Loss: 0.025491
Train Epoch: 1 [13000/15404 (84%)]	Loss: 0.025351
Train Epoch: 1 [14000/15404 (90%)]	Loss: 0.024657
Train Epoch: 1 [15000/15404 (97%)]	Loss: 0.023609

Train accuracy: 20.267462996624253

Train set: Average loss: 3.2591146041273076

Test set: Average loss: 2.076095271314311

Test accuracy: 24.627450980392158
Train Epoch: 2 [0/15404 (0%)]	Loss: 0.023766
Train Epoch